In [1]:
import json
import xlsxwriter

In [2]:
import pandas as pd

In [3]:
def next_alpha(s):
    return chr((ord(s.upper())+1 - 65) % 26 + 65)

def creat_lis_name(name):
    return [name, "not " + name]

def creta_matriz_confusao(wk, local, mat, ls_cat):
    def creat_name():
        letter = local[0]
        for i, cat in enumerate(ls_cat):
            wk.write(letter + str(local[1]+i), cat)


        for i, cat in enumerate(ls_cat):
            letter = next_alpha(letter)
            wk.write(letter + str(local[1]-1), cat)
    
    def matriz_confusao():
        last_c = 0
        letter = next_alpha(local[0])
        for linha in mat:
            for i,celula in enumerate(linha):
                wk.write(letter + str(local[1]+i), celula)
            letter = next_alpha(letter)
        
        last_c = local[1] + len(mat[0])
        return letter, last_c
     
    creat_name()
    letter = matriz_confusao()
    return letter

def set_descricao(wk, local, mat):
    wk.write(local[0] + str(local[1]-1), 'observada')
    wk.write(local[0] + str(local[1]), mat['observada'])
    local = (next_alpha(local[0]), local[1])

    wk.write(local[0] + str(local[1]-1), 'acaso')
    wk.write(local[0] + str(local[1]), mat['acaso'])
    local = (next_alpha(local[0]), local[1])

    wk.write(local[0] + str(local[1]-1), 'kappa')
    wk.write(local[0] + str(local[1]), mat['kappa'])
    local = (next_alpha(local[0]), local[1])

    wk.write(local[0] + str(local[1]-1), 'vies')
    wk.write(local[0] + str(local[1]), mat['vies'])
    local = (next_alpha(local[0]), local[1])

    wk.write(local[0] + str(local[1]-1), 'prevalencia')
    wk.write(local[0] + str(local[1]), mat['prevalencia'])
    local = (next_alpha(local[0]), local[1])
    return local


In [4]:
def create_cate(local_inicio, data, numero):
    mat = data['medido']['list_kappa_cat'][numero]['matriz_concordancia']
    ls_name = creat_lis_name(data['medido']['list_kappa_cat'][numero]['categoria'])

    
    la            = set_text(worksheet, (local_inicio[0], local_inicio[1]+4), "Measured")
    last_letter_4 = creta_matriz_confusao(worksheet, (local_inicio[0], local_inicio[1]+4), mat, ls_name)
    last_letter_5 = set_descricao(worksheet, (local_inicio[0], last_letter_4[1]+2) , data['medido']['list_kappa_cat'][numero])


    ## create maximum matrix
    la            = set_text(worksheet, 
                             (next_alpha(next_alpha(next_alpha(last_letter_4[0]))), local_inicio[1]+4),
                             "Maximum")
    last_letter_6 = creta_matriz_confusao(worksheet, 
                                          (next_alpha(next_alpha(next_alpha(last_letter_4[0]))), local_inicio[1]+4),
                                          data['medido']['list_kappa_cat_max'][numero]['matriz_concordancia'], ls_name)


    last_letter_7 = set_descricao(worksheet, 
                                (next_alpha(next_alpha(next_alpha(last_letter_4[0]))),last_letter_6[1]+2), 
                                  data['medido']['list_kappa_cat_max'][numero])
    
    return last_letter_7


In [5]:
def get_descritores_max(r_max, data):
    def map_categ(data, descritor, r_max):
        def map_des(linha):
            if r_max:
                return linha['catalogo_var_max'][descritor]
            else:
                return linha['catalogo_var'][descritor]

        return list(map(map_des, data['varios_kappa'] ))
    
    ls_descritores = list(data['varios_kappa'][0]['catalogo_var'].keys())
    ls = {}

    for descritor in ls_descritores:
        r_calcula = descritor != "matriz_concordancia"
        if r_calcula:
            ls[descritor] = map_categ(data, descritor, r_max)
    
    return ls


In [6]:
f = open('parsed.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
data

{'medido': {'et1': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2],
  'et2': [1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 2],
  'catalogo_var': {'categoria': 'Catalog',
   'observada': 0.384615,
   'acaso': 0.325444,
   'vies': 6,
   'kappa': 0.0877193,
   'prevalencia': 2.64575,
   'matriz_concordancia': [[2, 2, 0], [4, 0, 0], [0, 2, 3]]},
  'catalogo_var_max': {'categoria': 'Catalog',
   'observada': 0.846154,
   'acaso': 0.325444,
   'vies': 2,
   'kappa': 0.77193,
   'prevalencia': 1,
   'matriz_concordancia': [[4, 0, 0], [0, 4, 0], [2, 0, 3]]},
  'list_kappa_cat': [{'categoria': 'swimming',
    'observada': 0.454545,
    'acaso': 0.487603,
    'vies': 0.181818,
    'kappa': -0.0645162,
    'prevalencia': 0.0909091,
    'matriz_concordancia': [[2, 2], [4, 3]]},
   {'categoria': 'climbimg',
    'observada': 0.384615,
    'acaso': 0.573965,
    'vies': 0,
    'kappa': -0.444445,
    'prevalencia': 0.384615,
    'matriz_concordancia': [[0, 4], [4, 5]]},
   {'categoria': 'Immobility',
    'observada':

In [7]:
def set_text(wk, local, text):
    wk.write(local[0] + str(local[1]-2), text)
    return (local[0], local[1])

In [8]:
workbook = xlsxwriter.Workbook('demo.xlsx')
worksheet = workbook.add_worksheet()

mat  = data['medido']['catalogo_var']['matriz_concordancia']
ls_cat = data['ls_cat_txt']

local = ("C", 10)

la            = set_text(worksheet, local, "Measured")
last_letter   = creta_matriz_confusao(worksheet, local, mat, ls_cat)
last_letter_1 = set_descricao(worksheet, (local[0], last_letter[1]+2) , data['medido']['catalogo_var'])


la            = set_text(worksheet, (next_alpha(next_alpha(last_letter[0])), local[1]), "Maximum")
last_letter_2 = creta_matriz_confusao(worksheet, (next_alpha(next_alpha(last_letter[0])), local[1]),
                                      data['medido']['catalogo_var_max']['matriz_concordancia'], ls_cat)

last_letter_3 = set_descricao(worksheet, (next_alpha(next_alpha(last_letter[0])), last_letter_2[1]+2) , data['medido']['catalogo_var_max'])


In [9]:
last_letter

('G', 13)

In [10]:
last_letter

('G', 13)

In [11]:
# local = (last_letter[0], 10)
# mat  = data['medido']['catalogo_var']

# set_descricao(worksheet, local, mat)

In [12]:




# worksheet.write(local[0] + str(local[1]-1), 'observada')
# worksheet.write(local[0] + str(local[1]), mat['observada'])
# local = (next_alpha(local[0]), local[1])

# worksheet.write(local[0] + str(local[1]-1), 'acaso')
# worksheet.write(local[0] + str(local[1]), mat['acaso'])
# local = (next_alpha(local[0]), local[1])

# worksheet.write(local[0] + str(local[1]-1), 'kappa')
# worksheet.write(local[0] + str(local[1]), mat['kappa'])
# local = (next_alpha(local[0]), local[1])

# worksheet.write(local[0] + str(local[1]-1), 'vies')
# worksheet.write(local[0] + str(local[1]), mat['vies'])
# local = (next_alpha(local[0]), local[1])

# worksheet.write(local[0] + str(local[1]-1), 'prevalencia')
# worksheet.write(local[0] + str(local[1]), mat['prevalencia'])
# local = (next_alpha(local[0]), local[1])



In [13]:
local


('C', 10)

In [14]:
last_letter_7 = (local[0], last_letter_3[1])
for i in range(len(data['medido']['list_kappa_cat'])):
    last_letter_7 = create_cate((local[0], last_letter_7[1]), data, i)


In [15]:
# last_letter_7 = (local[0], last_letter_3[1])

In [16]:
last_letter_7

('N', 39)

In [17]:
data['varios_kappa'][0]['catalogo_var']

{'categoria': 'Catalog',
 'observada': 0.384615,
 'acaso': 0.360947,
 'vies': 6.55744,
 'kappa': 0.037037,
 'prevalencia': 2.64575,
 'matriz_concordancia': [[3, 2, 0], [5, 0, 0], [0, 1, 2]]}

In [18]:
dt = pd.DataFrame.from_dict(get_descritores_max(False, data))
dt_max = pd.DataFrame.from_dict(get_descritores_max(True, data))

In [19]:
# def set_text(wk, local, text):
dt_max
    
    

,categoria,observada,acaso,vies,kappa,prevalencia
0,Catalog,0.769231,0.360947,2.64575,0.638889,2.64575
1,Catalog,0.692308,0.301775,3.46410,0.559322,0.00000
2,Catalog,0.615385,0.284024,4.35890,0.462810,2.00000
3,Catalog,0.769231,0.349112,2.64575,0.645455,3.60555
4,Catalog,0.769231,0.325444,2.64575,0.657895,1.00000
5,Catalog,0.692308,0.313609,3.46410,0.551724,1.73205
6,Catalog,0.692308,0.301775,3.60555,0.559322,1.73205
7,Catalog,0.538462,0.349112,5.29150,0.290909,3.60555
8,Catalog,0.692308,0.360947,4.00000,0.518519,4.58258
9,Catalog,0.923077,0.390533,1.00000,0.873786,3.46410


In [20]:
# def get_descritores_max(r_max, data):
#     def map_categ(data, descritor, r_max):
#         def map_des(linha):
#             if r_max:
#                 return linha['catalogo_var_max'][descritor]
#             else:
#                 return linha['catalogo_var'][descritor]

#         return list(map(map_des, data['varios_kappa'] ))
    
#     ls_descritores = list(data['varios_kappa'][0]['catalogo_var'].keys())
#     ls = {}

#     for descritor in ls_descritores:
#         r_calcula = descritor != "matriz_concordancia"
#         if r_calcula:
#             ls[descritor] = map_categ(data, descritor, r_max)
    
#     return ls

    

In [21]:
def set_wt(wk, local, linha):
    wk.write(local[0] + str(local[1]), linha.mean())
    wk.write(next_alpha(local[0]) + str(local[1]), linha.std())
    return (next_alpha(local[0]), local[1])

def set_var(wk, local, nome):
    wk.write(local[0] + str(local[1]), nome)
    return local

def creat_cabecalho(wk, local):
    saida = set_var(wk, local, 'observada mean')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'observada std')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'acaso mean')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'acaso std')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'vies mean')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'vies std')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'prevalencia mean')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'prevalencia std')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'kappa mean')
    saida = set_var(wk, (next_alpha(saida[0]), saida[1]), 'kappa std')
    



In [22]:
last_letter_8 = (local[0], last_letter_7[1] + 4)

s     = creat_cabecalho(worksheet, (next_alpha(last_letter_8[0]), last_letter_8[1] - 1 ))
saida = set_var(worksheet, last_letter_8, "Bootstrap measured")
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt['observada'])
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt['acaso'])
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt['vies'])
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt['prevalencia'])
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt['kappa'])


saida = set_var(worksheet, (last_letter_8[0], last_letter_8[1]+1), "Bootstrap max")
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt_max['observada'])
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt_max['acaso'])
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt_max['vies'])
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt_max['prevalencia'])
saida = set_wt(worksheet, (next_alpha(saida[0]), saida[1] ), dt_max['kappa'])





In [23]:
# dt['observada'].mean()
# dt['observada'].std()

In [24]:
# dt['observada'].std()

In [39]:
dt['kappa']

0    0.037037
1    0.118644
2   -0.074380
3   -0.063636
4    0.087719
5   -0.120690
6    0.228814
7   -0.418182
8    0.518519
9   -0.135922
Name: kappa, dtype: float64

In [25]:
# def create_cate(local_inicio, data, numero):
#     mat = data['medido']['list_kappa_cat'][numero]['matriz_concordancia']
#     ls_name = creat_lis_name(data['medido']['list_kappa_cat'][numero]['categoria'])


#     last_letter_4 = creta_matriz_confusao(worksheet, (local[0], local_inicio[1]+4), mat, ls_name)
#     last_letter_5 = set_descricao(worksheet, (local[0], last_letter_4[1]+2) , data['medido']['list_kappa_cat'][numero])


#     last_letter_6 = creta_matriz_confusao(worksheet, 
#                                           (next_alpha(next_alpha(next_alpha(last_letter_4[0]))), local_inicio[1]+4),
#                                           data['medido']['list_kappa_cat_max'][numero]['matriz_concordancia'], ls_name)


#     last_letter_7 = set_descricao(worksheet, 
#                                 (next_alpha(next_alpha(next_alpha(last_letter_4[0]))),last_letter_6[1]+2), 
#                                   data['medido']['list_kappa_cat_max'][numero])
    
#     return last_letter_7

In [26]:


# mat = data['medido']['list_kappa_cat'][0]['matriz_concordancia']
# ls_name = creat_lis_name(data['medido']['list_kappa_cat'][0]['categoria'])


# last_letter_4 = creta_matriz_confusao(worksheet, (local[0], last_letter_3[1]+4), mat, ls_name)
# last_letter_5 = set_descricao(worksheet, (local[0], last_letter_4[1]+2) , data['medido']['list_kappa_cat'][0])


# last_letter_6 = creta_matriz_confusao(worksheet, 
#                                       (next_alpha(next_alpha(next_alpha(last_letter_4[0]))), last_letter_3[1]+4),
#                                       data['medido']['list_kappa_cat_max'][0]['matriz_concordancia'], ls_name)


# last_letter_7 = set_descricao(worksheet, 
#                             (next_alpha(next_alpha(next_alpha(last_letter_4[0]))),last_letter_6[1]+2), 
#                               data['medido']['list_kappa_cat_max'][0])




In [27]:
# last_letter_4

In [28]:
# last_letter_5

In [29]:
# (local[0], last_letter_3[1]+3)

In [30]:
data['medido']['list_kappa_cat'][0]

{'categoria': 'swimming',
 'observada': 0.454545,
 'acaso': 0.487603,
 'vies': 0.181818,
 'kappa': -0.0645162,
 'prevalencia': 0.0909091,
 'matriz_concordancia': [[2, 2], [4, 3]]}

In [31]:
data['medido']['list_kappa_cat_max'][0]

{'categoria': 'swimming',
 'observada': 0.818182,
 'acaso': 0.487603,
 'vies': 0.181818,
 'kappa': 0.645161,
 'prevalencia': 0.0909091,
 'matriz_concordancia': [[4, 0], [2, 5]]}

In [32]:
# data['medido']['catalogo_var_max']



In [33]:
workbook.close()

In [34]:



# Create an new Excel file and add a worksheet.
# workbook = xlsxwriter.Workbook('demo.xlsx')
# worksheet = workbook.add_worksheet()
# worksheet.set_column('A:A', 20)
# worksheet.write('A1', 'Hello')
# workbook.close()

In [35]:
# workbook = xlsxwriter.Workbook('demo.xlsx')
# worksheet = workbook.add_worksheet()

In [36]:
# local = ("A", 10)
# letter = local[0]
# for i, cat in enumerate(data['ls_cat_txt']):
#     worksheet.write(letter + str(local[1]+i), cat)

    
# for i, cat in enumerate(data['ls_cat_txt']):
#     letter = next_alpha(letter)
#     worksheet.write(letter + str(local[1]-1), cat)
    

In [37]:
# mat  = data['medido']['catalogo_var']['matriz_concordancia']
# local = ("B", 10)

In [38]:
# # def desenha_matriz_confusao(matriz, local):

# letter = local[0]
# for linha in mat:
#     for i,celula in enumerate(linha):
#         worksheet.write(letter + str(local[1]+i), celula)
    
#     letter = next_alpha(letter)
    
# workbook.close()